# Python's Functions Are First-Class

In [79]:
def get_speak_func(volume):
    def whisper(text):
        return text.lower() + '...'

    def yell(text):
        return text.upper() + '!'

    if volume > 0.5:
        return yell
    else:
        return whisper

Functions that do this are called **lexical closures** (or just **closures**, for short). A closure remembers the values from its enclosing lexical scope even when the program flow is no longer in that scope.

In practical terms, this means not only can functions return behaviors but they can also pre-configure those behaviors. Here’s another barebones example to illustrate this idea:

In [80]:
def get_speak_func_closure(text, volume):
    def whisper():
        return text.lower() + '...'

    def yell():
        return text.upper() + '!'

    if volume > 0.5:
        return yell
    else:
        return whisper

In [81]:
get_speak_func_closure('hello world', 0.7)()

'HELLO WORLD!'

Take a good look at the inner functions whisper and yell now. Notice how they no longer have a `text` parameter? But somehow they can
still access the `text` parameter defined in the parent function. In fact, they seem to **_capture_** and “remember” the value of that argument.

Functions that do this are called **_lexical closures_** (or just **_closures_**, for short). A closure remembers the values from its enclosing lexical scope even when the program flow is no longer in that scope.

In practical terms, this means not only can functions **return behaviors** but they can also **pre-configure** those behaviors. Here’s another barebones example to illustrate this idea:

In [82]:
def make_adder(n):
    def add(x):
        return x + n
    return add

In [83]:
plus_3 = make_adder(3)
plus_5 = make_adder(5)

In [84]:
plus_3(4)

7

In [85]:
plus_5(4)

9

In this example, make_adder serves as a **_factory_** to create and configure "adder” functions. Notice how the “adder” functions can still access the `n` argument of the `make_adder` function (the enclosing scope).

## Objects Can Behave Like Functions

Objects can be made **_callable_** by using `__call__` method

In [86]:
class Adder:
    def __init__(self, n):
        self.n = n
    def __call__(self, x):
        return self.n + x

In [87]:
plus_3 = Adder(3)
plus_3(4)

7

In [88]:
callable(plus_3)

True

# Lambdas Are Single-Expression Functions

In [89]:
add = lambda x, y: x + y
add(5, 3)

8

In [90]:
tuples = [(1, 'd'), (2, 'b'), (4, 'a'), (3, 'c')]
sorted(tuples, key=lambda x: x[1])

[(4, 'a'), (2, 'b'), (3, 'c'), (1, 'd')]

In [91]:
def make_adder(n):
    return lambda x: x + n

In [92]:
plus_3 = make_adder(3)
plus_5 = make_adder(5)

In [93]:
plus_3(4)

7

# The Power of Decorators

## Decorators Can Modify Behavior
Instead of simply returning the input function like the null decorator did, this uppercase decorator defines a new function on the fly (a closure) and uses it to wrap the input function in order to modify its behavior at call time.

In [94]:
def greet():
    return 'Hello!'

In [95]:
def null_decorator(func):
    return func

In [96]:
dec_greet = null_decorator(greet)
dec_greet()

'Hello!'

In [97]:
def uppercase(func):
    def wrapper():
        original_result = func()
        modified_result = original_result.upper()
        return modified_result
    return wrapper

In [98]:
correct = uppercase(greet)
correct()

'HELLO!'

In [99]:
greet

<function __main__.greet()>

In [100]:
null_decorator(greet)

<function __main__.greet()>

In [101]:
uppercase(greet)

<function __main__.uppercase.<locals>.wrapper()>

In [102]:
def my_upper(func):
    original_result = func()
    return original_result.upper()

In [103]:
my_upper(greet)

'HELLO!'

In [104]:
@uppercase
def greet():
    return 'Hello!'

In [105]:
greet()

'HELLO!'

## Decorating Functions That Accept Arguments

In [106]:
def trace(func):
    def wrapper(*args, **kwargs):
        print(f'TRACE: calling {func.__name__}() '
              f'with {args}, {kwargs}')
        
        original_result = func(*args, **kwargs)
        
        print(f'TRACE: {func.__name__}() '
              f'returned {original_result!r}')
        return original_result
    return wrapper

In [107]:
@trace
def say(name, line):
    return f'{name}: {line}'

In [108]:
say('Jane', 'Hello, World')

TRACE: calling say() with ('Jane', 'Hello, World'), {}
TRACE: say() returned 'Jane: Hello, World'


'Jane: Hello, World'

## How to Write "Debuggable" Decorators

In [114]:
def greet():
    """Return a friendly greeting."""
    return 'Hello~'

decorated_greet = uppercase(greet)

In [110]:
greet.__name__

'greet2'

In [111]:
decorated_greet.__name__

'wrapper'

In [112]:
decorated_greet.__doc__

In [116]:
import functools

def uppercase(func):
    @functools.wraps(func)
    def wrapper():
        return func().upper()
    return wrapper

@uppercase
def greet():
    """Return a friendly greeing."""
    return 'Hello~'

In [117]:
greet.__name__

'greet'

# Fun With `*args` and `**kwargs`
(optional and keyword arguments)

## Forwarding Optional or Keyword Arguments

In [113]:
def foo(x, *args, **kwargs):
    kwargs['name'] = 'Alice'
    new_args = args + ('extra', )
    bar(x, *new_args, **kwargs)

This technique can be useful for subclassing and writing wrapper functions.For example, you can use it to extend the behavior of a parent class without having to replicate the full signature of its constructor in the child class. This can be quite convenient if you’re working with an API that might change outside of your control:

### Usage 1. subclassing
* modifying or overriding behavior in some external class which you don't control 
* The downside here is that the constructor now has a rather unhelpful signature

In [118]:
class Car:
    def __init__(self, color, mileage):
        self.color = color
        self.mileage = mileage
        
class AlwaysBlueCar(Car):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.color = 'blue'

In [120]:
AlwaysBlueCar('green', 48392).color

'blue'

### Usage 2 Wrapper functions

In [121]:
def trace(f):
    @functools.wraps(f)
    def decorated_function(*args, **kwargs):
        print(f, args, kwargs)
        result = f(*args, **kwargs)
        print(result)
    return decorated_function
    
@trace
def greet(greeting, name):
    return '{}, {}!'.format(greeting, name)

With techniques like this one, it's sometimes difficult to balance the idea of making your code explicit enough and yet adhere to the Don't Repeat Yourself (DRY) principle. This will always be a tough choice to make.

# Function Argument Unpacking

In [125]:
def print_vector(x, y, z):
    print('<%s, %s, %s>' % (x, y, z))

In [123]:
tuple_vec = (1, 0, 1)
list_vec = [1, 0, 1]

In [131]:
print_vector(*tuple_vec)
print_vector(*list_vec)

<1, 0, 1>
<1, 0, 1>


In [128]:
genexpr = (x * x for x in range(3))
print_vector(*genexpr)

<0, 1, 4>


In [129]:
dict_vec = {'y': 0, 'z': 1, 'x': 1}
print_vector(**dict_vec)
print_vector(*dict_vec)

<1, 0, 1>
<y, z, x>
